In [ ]:
import implicit
import pandas as pd
import numpy
from scipy import sparse
from sklearn.externals import joblib

In [ ]:
%time df = pd.read_csv('interactions.csv', usecols = ['customerid','productid','amount'], dtype = {'customerid':"category", 'productid':"category", "amount":int})

In [ ]:
df['amount'].replace(1,5)
df['amount'].replace(0,1)

%time grouped = df.groupby(['customerid','productid'])['amount'].sum().reset_index()

In [ ]:
rowdata = grouped['productid'].cat.codes
coldata = grouped['customerid'].cat.codes
data = grouped['amount']

orders = sparse.coo_matrix((data, (rowdata, coldata)))
print(repr(orders))

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=5)
%time model.fit(orders)

In [ ]:
orders_transposed = orders.T.tocsr()
userid = grouped['customerid'].cat.categories.get_loc('31460292')

recommendations = model.recommend(userid, orders_transposed, N=10, filter_already_liked_items=True)

usercategories = grouped['customerid'].cat;
itemcategories = grouped['productid'].cat;

print('Reco items')
for itemid,score in recommendations:
    print("%s / %s" % (itemcategories.categories[itemcategories.codes[itemid]], score))
    
print('Similar users')    
for uid,score in model.similar_users(userid): 
    print("%s / %s" % (usercategories.categories[usercategories.codes[uid]], score))